<a href="https://colab.research.google.com/github/ToumaTanaka/Data_Science/blob/main/Mathematical_Optimization/Allocation_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 割当問題
* 0-1整数計画問題として解く(解ベクトルxの各要素を0または1のみに限定したもの)
* PythonのソルバーであるPuLPを利用する
* 割り当てた時の目的関数が最小になる組み合わせを探す

In [ ]:
!pip install pulp

     |████████████████████████████████| 14.2 MB 4.9 MB/s 


In [ ]:
import pulp
import pandas as pd
from google.colab import files

In [ ]:
#shabanを配送ルート、jomuin_codeをその日の出勤ドライバーとする
#scoreは最小化したい値

df = pd.read_csv('data.csv')
df.head()

,Unnamed: 0,shaban,jomuin_code,score
0,0,1,1,69
1,1,1,2,67
2,2,1,3,86
3,3,1,4,70
4,4,1,5,88


In [ ]:
df['Var'] = [pulp.LpVariable(f'x{df.shaban[L]}_{df.jomuin_code[L]}',cat="Binary") for L in df.index]
df

,Unnamed: 0,shaban,jomuin_code,score,Var
0,0,1,1,69,x1_1
1,1,1,2,67,x1_2
2,2,1,3,86,x1_3
3,3,1,4,70,x1_4
4,4,1,5,88,x1_5
...,...,...,...,...,...
145,145,10,11,63,x10_11
146,146,10,12,90,x10_12
147,147,10,13,71,x10_13
148,148,10,14,69,x10_14


In [ ]:
#最小値探索のモデルを定義
problem = pulp.LpProblem('割り当て問題', sense=pulp.LpMinimize)

#制約条件

#同じ配送ルートごとにデータをまとめる
#iには配送ルートの番号が格納、vには同じ配送ルートの配列が格納されている
#一つの配送ルートに必ず一人を割り当てる制約
for i, v in df.groupby('shaban'):
  problem += pulp.lpSum(v.Var) == 1
  


#同じドライバーごとにデータをまとめる
#jにはドライバーの番号が格納、vには同じドライバーの配列が格納されている
#一人のドライバーに一つ割り当てるか、または一つも割り当てない制約
for j, v in df.groupby('jomuin_code'):
  problem += pulp.lpSum(v.Var) <= 1


#定義したモデルに目的関数を追加
#df.scoreとdf.Varの内積
problem += pulp.lpDot(df.score,df.Var)

In [ ]:
#解を求める
result = problem.solve()

#現在のstatusを表示
#-3: 'Undefined',    未定義
#-2: 'Unbounded',   非有界
#-1: 'Infeasible',     実行不可能
#0: 'Not Solved'    解けなかった
#1: 'Optimal'       最適解を発見
print(pulp.LpStatus[result])

#目的関数の値を表示
print(pulp.value(problem.objective))

#最適化結果をデータフレームに追加
df['Val'] = df.Var.apply(pulp.value)

Optimal
526.0


In [ ]:
df.head()

,Unnamed: 0,shaban,jomuin_code,score,Var,Val
0,0,1,1,69,x1_1,0.0
1,1,1,2,67,x1_2,0.0
2,2,1,3,86,x1_3,0.0
3,3,1,4,70,x1_4,0.0
4,4,1,5,88,x1_5,0.0


In [ ]:
df_s = df[df.Val > 0][['shaban', 'jomuin_code', 'score']]
df_s

,shaban,jomuin_code,score
5,1,6,53
26,2,12,50
40,3,11,52
59,4,15,54
66,5,7,53
87,6,13,51
92,7,3,58
109,8,5,50
129,9,10,55
143,10,9,50


In [ ]:
#結果の出力
df_s.to_csv("optimal.csv")
files.download("optimal.csv") 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>